In [1]:
import json
import pandas as pd
import re


In [2]:
nodes = pd.read_csv('output-data/nodes_sankey.csv')
nodes


,LABEL_NODE,ID,FUNCTION,JOIN_ARG,NAME_NODE,FILTER,COLOR
0,User::CurrentDateTime,0,Variable,NaN,User::CurrentDateTime,NaN,green
1,User::Errors_lookup,1,Variable,NaN,User::Errors_lookup,NaN,green
2,User::Rows_suppliers,2,Variable,NaN,User::Rows_suppliers,NaN,green
3,User::SQL_LA_table,3,Variable,NaN,User::SQL_LA_table,NaN,green
4,Merge and filter@Alter name pref,4,DerivedColumn,NaN,Alter name pref,NaN,black
5,Merge and filter@Determine rows sup,5,RowCount,NaN,Determine rows sup,NaN,black
6,Merge and filter@Load supplier,6,SSISODBCSrc,NaN,Load supplier,NaN,black
7,dbo.Suppliers_Extract,7,DataSources,NaN,dbo.Suppliers_Extract,NaN,black
8,dbo.Products_Extract,8,DataSources,NaN,dbo.Products_Extract,NaN,black
9,Merge and filter@Match with products,9,Lookup,SupplierID = SupplierID,Match with products,NaN,dodgerblue


In [3]:
nodes_order = pd.read_csv('output-data/nodes-order.csv')
nodes_order


,Unnamed: 0,ID_block_out,ID_block_in,type_block_out,type_block_in
0,0,Package\Merge and filter\Split preferences,Package\Merge and filter\Alter name pref,Microsoft.ConditionalSplit,Microsoft.DerivedColumn
1,1,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers,Microsoft.ConditionalSplit,Microsoft.UnionAll
2,2,Package\Merge and filter\Alter name pref,Package\Merge and filter\Union suppliers,Microsoft.DerivedColumn,Microsoft.UnionAll
3,3,Package\Merge and filter\Match with products,Package\Merge and filter\Split preferences,Microsoft.Lookup,Microsoft.ConditionalSplit
4,4,Package\Merge and filter\Load supplier,Package\Merge and filter\Determine rows sup,Microsoft.SSISODBCSrc,Microsoft.RowCount
5,5,Package\Merge and filter\Determine rows sup,Package\Merge and filter\Match with products,Microsoft.RowCount,Microsoft.Lookup
6,6,Package\Merge and filter\Union suppliers,Package\Merge and filter\Write Update sup table,Microsoft.UnionAll,Microsoft.SSISODBCDst


In [5]:
lineages = pd.read_csv('output-data/lineages.csv')
lineages[50:-50]
lineages[:50]
#lineages[-50:]


,Unnamed: 0,column_in,column_out,TRANSFORMATION,SOURCE_FIELD,TARGET_FIELD,SOURCE_NODE,TARGET_NODE,LINK_VALUE,ROW_ID,COLOR,ID,LABEL_NODE
0,0,dbo.Suppliers_Extract[ValidFrom],Package\Merge and filter\Load supplier[ValidFrom],NaN,ValidFrom,ValidFrom,7,6,1,0,aliceblue,6,Merge and filter@Load supplier
1,1,dbo.Suppliers_Extract[ValidTo],Package\Merge and filter\Load supplier[ValidTo],NaN,ValidTo,ValidTo,7,6,1,1,aliceblue,6,Merge and filter@Load supplier
2,2,dbo.Suppliers_Extract[Active],Package\Merge and filter\Load supplier[Active],NaN,Active,Active,7,6,1,2,aliceblue,6,Merge and filter@Load supplier
3,3,dbo.Suppliers_Extract[Country],Package\Merge and filter\Load supplier[Country],NaN,Country,Country,7,6,1,3,aliceblue,6,Merge and filter@Load supplier
4,4,dbo.Suppliers_Extract[SupplierID],Package\Merge and filter\Load supplier[Supplie...,NaN,SupplierID,SupplierID,7,6,1,4,aliceblue,6,Merge and filter@Load supplier
5,5,dbo.Suppliers_Extract[CompanyName],Package\Merge and filter\Load supplier[Company...,NaN,CompanyName,CompanyName,7,6,1,5,aliceblue,6,Merge and filter@Load supplier
6,6,dbo.Suppliers_Extract[Preferred],Package\Merge and filter\Load supplier[Preferred],NaN,Preferred,Preferred,7,6,1,6,aliceblue,6,Merge and filter@Load supplier
7,7,dbo.Suppliers_Extract[Discount],Package\Merge and filter\Load supplier[Discount],NaN,Discount,Discount,7,6,1,7,aliceblue,6,Merge and filter@Load supplier
8,8,dbo.Suppliers_Extract[Address],Package\Merge and filter\Load supplier[Address],NaN,Address,Address,7,6,1,8,aliceblue,6,Merge and filter@Load supplier
9,9,dbo.Suppliers_Extract[Region],Package\Merge and filter\Load supplier[Region],NaN,Region,Region,7,6,1,9,aliceblue,6,Merge and filter@Load supplier
